In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from collections import Counter
torch.__version__

'1.3.0'

# 5.2 Pytorch处理结构化数据
## 简介
在介绍之前，我们首先要明确下什么是结构化的数据。结构化数据，可以从名称中看出，是高度组织和整齐格式化的数据。它是可以放入表格和电子表格中的数据类型。对我们来说，结构化数据可以理解为就是一张2维的表格，例如一个csv文件，就是结构化数据，在英文一般被称作Tabular Data或者叫 structured data，下面我们来看一下结构化数据的例子。

一下文件来自于fastai的自带数据集：
https://github.com/fastai/fastai/blob/master/examples/tabular.ipynb
fastai样例在这里


## 数据预处理
我们拿到的结构化数据，一般都是一个csv文件或者数据库中的一张表格，所以对于结构化的数据，我们直接使用pasdas库处理就可以了

In [2]:
#读入文件
df = pd.read_csv('./data/adult.csv')
#salary是这个数据集最后要分类的结果
df['salary'].unique()

array(['>=50k', '<50k'], dtype=object)

In [3]:
#查看数据类型
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [4]:
#pandas的describe可以告诉我们整个数据集的大概结构，是非常有用的
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32074.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.079815,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572999,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [5]:
#查看一共有多少数据
len(df)

32561

对于模型的训练，只能够处理数字类型的数据，所以这里面我们首先要将数据分成三个类别
- 训练的结果标签：即训练的结果，通过这个结果我们就能够明确的知道我们这次训练的任务是什么，是分类的任务，还是回归的任务。
- 分类数据：这类的数据是离散的，无法通过直接输入到模型中进行训练，所以我们在预处理的时候需要优先对这部分进行处理，这也是数据预处理的主要工作之一
- 数值型数据：这类数据是直接可以输入到模型中的，但是这部分数据有可能还是离散的，所以如果需要也可以对其进行处理，并且处理后会对训练的精度有很大的提升，这里暂且不讨论

In [6]:
#训练结果
result_var = 'salary'
#分类型数据
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race','sex','native-country']
#数值型数据
cont_names = ['age', 'fnlwgt', 'education-num','capital-gain','capital-loss','hours-per-week']

人工确认完数据类型后，我们可以看一下分类类型数据的数量和分布情况

In [7]:
for col in df.columns:
    if col in cat_names:
        ccol=Counter(df[col])
        print(col,len(ccol),ccol)
        print("\r\n")

workclass 9 Counter({' Private': 22696, ' Self-emp-not-inc': 2541, ' Local-gov': 2093, ' ?': 1836, ' State-gov': 1298, ' Self-emp-inc': 1116, ' Federal-gov': 960, ' Without-pay': 14, ' Never-worked': 7})


education 16 Counter({' HS-grad': 10501, ' Some-college': 7291, ' Bachelors': 5355, ' Masters': 1723, ' Assoc-voc': 1382, ' 11th': 1175, ' Assoc-acdm': 1067, ' 10th': 933, ' 7th-8th': 646, ' Prof-school': 576, ' 9th': 514, ' 12th': 433, ' Doctorate': 413, ' 5th-6th': 333, ' 1st-4th': 168, ' Preschool': 51})


marital-status 7 Counter({' Married-civ-spouse': 14976, ' Never-married': 10683, ' Divorced': 4443, ' Separated': 1025, ' Widowed': 993, ' Married-spouse-absent': 418, ' Married-AF-spouse': 23})


occupation 16 Counter({' Prof-specialty': 4073, ' Craft-repair': 4028, ' Exec-managerial': 4009, ' Adm-clerical': 3720, ' Sales': 3590, ' Other-service': 3247, ' Machine-op-inspct': 1968, ' ?': 1820, ' Transport-moving': 1566, ' Handlers-cleaners': 1347, ' Farming-fishing': 977, ' Tech

下一步就是要将分类型数据转成数字型数据，在这部分里面，我们还做了对于缺失数据的填充

In [8]:
for col in df.columns:
    if col in cat_names:
        df[col].fillna('---')
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    if col in cont_names:
        df[col]=df[col].fillna(0)

上面的代码中：

我们首先使用了pandas的fillna函数对分类的数据做了空值的填充，这里面标识成一个与其他现有值不一样的值就可以，这里面我使用的三个中划线 --- 作为标记，然后就是使用了sklearn的LabelEncoder函数进行了数据的处理

然后有对我们的数值型的数据做了0填充的处理，对于数值型数据的填充，也可以使用平均值，或者其他方式填充，这个不是我们的重点，就不详细说明了。

In [9]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,4,101320,7,12.0,2,15,5,4,0,0,1902,40,39,>=50k
1,44,4,236746,12,14.0,0,4,1,4,1,10520,0,45,39,>=50k
2,38,4,96185,11,0.0,0,15,4,2,0,0,0,32,39,<50k
3,38,5,112847,14,15.0,2,10,0,1,1,0,0,40,39,>=50k
4,42,6,82297,5,0.0,2,8,5,2,0,0,0,50,39,<50k


数据处理完成后可以看到，现在所有的数据都是数字类型的了，可以直接输入到模型进行训练了.

In [10]:
#分割下训练数据和标签
Y = df['salary']
Y_label = LabelEncoder()
Y=Y_label.fit_transform(Y)
Y

array([1, 1, 0, ..., 1, 0, 0])

In [11]:
X=df.drop(columns=result_var)
X

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,49,4,101320,7,12.0,2,15,5,4,0,0,1902,40,39
1,44,4,236746,12,14.0,0,4,1,4,1,10520,0,45,39
2,38,4,96185,11,0.0,0,15,4,2,0,0,0,32,39
3,38,5,112847,14,15.0,2,10,0,1,1,0,0,40,39
4,42,6,82297,5,0.0,2,8,5,2,0,0,0,50,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,36,4,297449,9,13.0,0,10,1,4,1,14084,0,40,39
32557,23,0,123983,9,13.0,4,0,3,3,1,0,0,40,39
32558,53,4,157069,7,12.0,2,7,0,4,1,0,0,40,39
32559,32,2,217296,11,9.0,2,14,5,4,0,4064,0,22,39


以上，基本的数据预处理已经完成了，上面展示的只是一些必要的处理，如果要提高训练准确率还有很多技巧，这里就不详细说明了。
## 定义数据集
要使用pytorch处理数据，肯定要使用Dataset进行数据集的定义，下面定义一个简单的数据集

In [12]:
class tabularDataset(Dataset):
    def __init__(self, X, Y):
        self.x = X#.to_numpy().astype(float)
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (self.x.values[idx], self.y[idx])

In [13]:
train_ds = tabularDataset(X, Y)

可以直接使用索引访问定义好的数据集中的数据

In [14]:
train_ds[0]

(array([4.9000e+01, 4.0000e+00, 1.0132e+05, 7.0000e+00, 1.2000e+01,
        2.0000e+00, 1.5000e+01, 5.0000e+00, 4.0000e+00, 0.0000e+00,
        0.0000e+00, 1.9020e+03, 4.0000e+01, 3.9000e+01]), 1)

## 定义模型
数据已经准备完毕了，下一步就是要定义我们的模型了，这里使用了3层线性层的简单模型作为处理

In [15]:
class tabularModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(14, 500)
        self.lin2 = nn.Linear(500, 100)
        self.lin3 = nn.Linear(100, 2)
        self.bn1 = nn.BatchNorm1d(14)
        self.bn2 = nn.BatchNorm1d(500)
        self.bn3 = nn.BatchNorm1d(100)
        

    def forward(self,x_in):
        #print(x_in.shape)
        x=x_in
        x = self.bn1(x)
        x = F.relu(self.lin1(x))
        #print(x)
        
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        #print(x)
        
        x = self.bn3(x)
        x = self.lin3(x)
        x=torch.sigmoid(x)
        return x

在定义模型的时候看到了我们加入了Batch Normalization来做批量的归一化：
批量归一化的内容请见这篇文章：https://mp.weixin.qq.com/s/FFLQBocTZGqnyN79JbSYcQ

或者扫描这个二维码，在微信中查看:
![](https://raw.githubusercontent.com/zergtant/pytorch-handbook/master/deephub.jpg)

## 训练

In [16]:
#训练前指定使用的设备
DEVICE=torch.device("cpu")
if torch.cuda.is_available():
        DEVICE=torch.device("cuda")
print(DEVICE)

cuda


In [17]:
#损失函数
criterion =nn.CrossEntropyLoss().to(DEVICE)

In [18]:
#实例化模型
model = tabularModel().to(DEVICE)
print(model)

tabularModel(
  (lin1): Linear(in_features=14, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=100, bias=True)
  (lin3): Linear(in_features=100, out_features=2, bias=True)
  (bn1): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [19]:
#测试模型是否没问题
rn=torch.rand(3,14).to(DEVICE)
model(rn)

tensor([[0.4717, 0.4703],
        [0.6462, 0.3824],
        [0.3931, 0.6696]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [25]:
#学习率
LEARNING_RATE=0.01
#BS
batch_size = 2048
#优化器
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [26]:
#DataLoader加载数据
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)

以上的基本步骤是每个训练过程都需要的，所以就不多介绍了，下面开始进行模型的训练

In [27]:
%%time
model.train()
#训练10轮
TOTAL_EPOCHS=10
#记录损失函数
losses = [];
for epoch in range(TOTAL_EPOCHS):
    for i, (x, y) in enumerate(train_dl):
        x = x.float().to(DEVICE) #输入必须未float类型
        y = y.long().to(DEVICE) #结果标签必须未long类型
        #清零
        optimizer.zero_grad()
        outputs = model(x)
        #计算损失函数
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().data.item());     
        print ('Epoch : %d/%d,   Loss: %.4f'%(epoch+1, TOTAL_EPOCHS, loss.data.item()))

Epoch : 1/10,   Loss: 0.7006
Epoch : 1/10,   Loss: 0.6112
Epoch : 1/10,   Loss: 0.5917
Epoch : 1/10,   Loss: 0.5557
Epoch : 1/10,   Loss: 0.5422
Epoch : 1/10,   Loss: 0.5292
Epoch : 1/10,   Loss: 0.5104
Epoch : 1/10,   Loss: 0.5031
Epoch : 1/10,   Loss: 0.5068
Epoch : 1/10,   Loss: 0.4939
Epoch : 1/10,   Loss: 0.4782
Epoch : 1/10,   Loss: 0.4864
Epoch : 1/10,   Loss: 0.4845
Epoch : 1/10,   Loss: 0.4787
Epoch : 1/10,   Loss: 0.4746
Epoch : 1/10,   Loss: 0.4671
Epoch : 2/10,   Loss: 0.4517
Epoch : 2/10,   Loss: 0.4780
Epoch : 2/10,   Loss: 0.4703
Epoch : 2/10,   Loss: 0.4614
Epoch : 2/10,   Loss: 0.4655
Epoch : 2/10,   Loss: 0.4640
Epoch : 2/10,   Loss: 0.4520
Epoch : 2/10,   Loss: 0.4710
Epoch : 2/10,   Loss: 0.4644
Epoch : 2/10,   Loss: 0.4633
Epoch : 2/10,   Loss: 0.4533
Epoch : 2/10,   Loss: 0.4676
Epoch : 2/10,   Loss: 0.4724
Epoch : 2/10,   Loss: 0.4544
Epoch : 2/10,   Loss: 0.4530
Epoch : 2/10,   Loss: 0.4597
Epoch : 3/10,   Loss: 0.4546
Epoch : 3/10,   Loss: 0.4642
Epoch : 3/10, 

训练完成后我们可以看一下模型的准确率

In [28]:
model.eval()
correct = 0
total = 0
for i,(x, y) in enumerate(train_dl):
    x = x.float().to(DEVICE)
    y = y.long()
    outputs = model(x).cpu()
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum()
print('准确率: %.4f %%' % (100 * correct / total))

准确率: 86.0000 %


通过基本的训练流程，准确率虽然达到86%了，但是损失在0.4就已经不下降了，说明这个网络最大程度就是这个水平了，那么还有什么办法提高准确程度呢？。后面还会介绍更高级的数据的处理方法，以提高准确程度